In [1]:
import os
import pickle
import numpy as np
import re
import emoji
import regex

In [2]:
def extract_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [3]:
def delete_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            
            text = re.sub("|".join(word), "", text)

    return text

In [4]:
# load emoji list
fullfile = os.path.expanduser("~/Dropbox/insight/Emoji/"+'mySmileys.p')
with open(fullfile, 'rb') as fp:
    emoji_list = pickle.load(fp)
print(emoji_list)
print(len(emoji_list))

['😀', '😁', '😂', '🤣', '😃', '😄', '😅', '😆', '😉', '😊', '😋', '😎', '😍', '😘', '😗', '😙', '😚', '☺️', '🙂', '🤗', '🤔', '😐', '😑', '😶', '🙄', '😏', '😣', '😥', '😮', '🤐', '😯', '😪', '😫', '😴', '😌', '😛', '😜', '😝', '🤤', '😒', '😓', '😔', '😕', '🙃', '🤑', '😲', '☹️', '🙁', '😖', '😞', '😟', '😤', '😢', '😭', '😦', '😧', '😨', '😩', '😬', '😰', '😱', '😳', '😵', '😡', '😠', '😷', '🤒', '🤕', '🤢', '🤧', '😇', '🤠', '🤡', '🤥', '🤓']
75


## sanitize tweets

In [5]:
def sanitize_tweets(tweet):
    # delete twitter-specific tags
    b1 = re.findall(r'@\S*', tweet) # Twitter user names
    b2 = re.findall(r'RT\S*', tweet) # "RT"
    b3 = re.findall(r'http\S*', tweet) # links

    c = tweet #list(tweet.split())
    for b in b1:
        c = re.sub(re.escape(b),"",c)
    for b in b2:
        c = re.sub(re.escape(b),"",c)
    for b in b3:
        c = re.sub(re.escape(b),"",c)
    return c
    
tweet = "RT @aa RT @bbb https:__ and other words! httpsaa" #RawT[-1]
sanitize_tweets(tweet)

'     and other words! '

In [6]:
# preprocessing with removing repeat tweets
# IN
# target_names = ['😍','😩','🙄'] # emoji_list # 
target_names = ['😍','😩','🙄']

# OUT
rawdata = []
data = []
raw_target = []
Len = []
filenames = []
target_arr = np.zeros((5000*75*2, 75)) # init bigger than actual
search_emoji = []

# main loop
ii = 0
for keyword in target_names:
    print(keyword)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
    with open(fullfile, 'rb') as fp:
        Tweets1 = pickle.load(fp)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k_batch2/"+'outfile'+keyword)
    with open(fullfile, 'rb') as fp:
        Tweets2 = pickle.load(fp)
    Tweets = Tweets1+Tweets2

    # remove repeat tweets
    myset = set(Tweets)
    Tweets_unique = list(myset)

    Text = []
    RawText = []
    E = []
    for tweet in Tweets_unique:
        # remove tweets with more than one smiley
        emojis = extract_emojis(tweet)
        a = set(emojis) & set(emoji_list)
        a2 = list(a)
        if len(a2)==1 and a2[0]==keyword: # else: deal with later if doing multi-D target

            # save raw tweet
            RawText.append(tweet)
            # save all emojis from input tweets
            emojis = extract_emojis(tweet)
            E.append(emojis)

    #         # save emojis into target array 
    #         for x in emojis:
    #             if x in emoji_list:
    #                 ix = Edict[x]
    #                 target_arr[ii,ix] = 1

            # save sanitized text
            text = sanitize_tweets(tweet)
            text = delete_emojis(text)
            Text.append(text)
            ii = ii+1

    data = data + Text
    rawdata = rawdata + RawText
    raw_target = raw_target + E
    filenames = filenames + [fullfile]
    search_emoji = search_emoji + [keyword]
    Len.append(len(Text))

😍
😩
🙄


In [7]:
# optional now: get 1-D target
numEmojis = len(target_names)
arr = []
for i in range(numEmojis):
    arr.extend([i] * Len[i]) 
target_1d = np.array(arr, dtype=int)

In [8]:
print(Len)

[4467, 4513, 5500]


In [9]:
# this struct is saving relatively raw data, for later use ~ visualizations
class tweet_data:
    pass

D = tweet_data()
D.raw_data = rawdata
D.data = data
D.raw_target = raw_target
D.filenames = filenames
D.numTweets = Len
D.target_1d = target_1d
# D.target_arr = target_arr
D.target_names = target_names

In [10]:
# 10/02
fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_data_3x_unique_singles_target1d.p')
# fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_data_75x_unique_singles_target1d.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(D, fp)

In [14]:
import pandas as pd
df = pd.DataFrame(data[:100])
df

,0
0,Absolutely stunning views of Ring-tail on #Hum...
1,Red sky at night... \n\nYou can't get better...
2,#AMAs⁠ ⁠ Favorite Social Artist I vote for BT...
3,Cheesy Marmite Waffles?! #FridayFeeling \n
4,This is the life.. my 2 beautiful girls and lo...
5,You're extra beautiful today... \n \n#maine...
6,#Thalapathy Morning Friends \n\n#Oruviralpur...
7,OMG she’s a doll!
8,Dayumm Under Armour never looked this good bef...
9,Your smile can change the world\n\n#TejasswiPr...
